<!DOCTYPE html>
<html>
<head>
    <style>
        table {
            width: 50%;
            border-collapse: collapse;
            margin: 20px auto;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 15px;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
<center>
<img src="https://hcmut.edu.vn/img/nhanDienThuongHieu/bk_name_en.png" alt="hcmut logo"/>

<table>
    <tr>
        <th>Role</th>
        <th>Name</th>
        <th>ID</th>
    </tr>
    <tr>
        <td>Institution</td>
        <td>Ho Chi Minh University of Technology</td>
        <td></td>
    </tr>
    <tr>
        <td>Mentor</td>
        <td>Dr. Nguyen Duc Dung</td>
        <td></td>
    </tr>
    <tr>
        <td>Actor</td>
        <td>Doan Tran Cao Tri</td>
        <td>2010733</td>
    </tr>
</table>

</center>
</body>
</html>


In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

### Setup and Import

In [2]:
!pip install datasets evaluate -q
!pip install jiwer -q
!pip install -U accelerate -q
!pip install -U transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("TeeA/Pokemon-Captioning-Classification")
dataset

Generating train split:   0%|          | 0/4869 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1390 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/732 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels', 'caption'],
        num_rows: 4869
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels', 'caption'],
        num_rows: 1390
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels', 'caption'],
        num_rows: 732
    })
})

select and rename features

In [5]:
for split in dataset.keys():
    dataset[split] = dataset[split].select_columns(['labels', 'caption']).rename_column('caption', 'text').rename_column('labels', 'label')

dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4869
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1390
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 732
    })
})

In [6]:
dataset["train"][0]

{'label': 57,
 'text': ' It has a yellow body with a tan belly and a tuft of fur on its head. It has three claws on each foot and a long tail with a tuft of fur on the end. Its eyes are red, and it has a small, triangular nose.'}

### Preprocessing

In [7]:
from transformers import AutoTokenizer

checkpoint = "TeeA/roberta-classifier-pokemon"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

RobertaTokenizerFast(name_or_path='TeeA/roberta-classifier-pokemon', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

### Base model

In [8]:
labels = dataset['train'].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'Hitmonlee'

In [9]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(checkpoint,
                                                      num_labels=len(labels),
                                                      id2label=id2label,
                                                      label2id=label2id
                                                      )
model

config.json:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [10]:
model.config.problem_type = "single_label_classification"
model.config

RobertaConfig {
  "_name_or_path": "TeeA/roberta-classifier-pokemon",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Porygon",
    "1": "Goldeen",
    "2": "Hitmonlee",
    "3": "Hitmonchan",
    "4": "Gloom",
    "5": "Aerodactyl",
    "6": "Mankey",
    "7": "Seadra",
    "8": "Gengar",
    "9": "Venonat",
    "10": "Articuno",
    "11": "Seaking",
    "12": "Dugtrio",
    "13": "Machop",
    "14": "Jynx",
    "15": "Oddish",
    "16": "Dodrio",
    "17": "Dragonair",
    "18": "Weedle",
    "19": "Golduck",
    "20": "Flareon",
    "21": "Krabby",
    "22": "Parasect",
    "23": "Ninetales",
    "24": "Nidoqueen",
    "25": "Kabutops",
    "26": "Drowzee",
    "27": "Caterpie",
    "28": "Jigglypuff",
    "29": "Machamp",
    "30": "Clefairy",
    "31": "K

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [12]:
tokenized_ds = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4869 [00:00<?, ? examples/s]

Map:   0%|          | 0/1390 [00:00<?, ? examples/s]

Map:   0%|          | 0/732 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer

# Training

In [ ]:
import torch
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer


# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):

#         outputs = model(
#             input_ids=inputs['input_ids'],
#             attention_mask=inputs['attention_mask'],
#         )
#         loss = F.cross_entropy(outputs['logits'].unsqueeze(0), inputs['labels'])
#         return (loss, outputs) if return_outputs else loss

# EVAL_STEPS = 1000

training_args = TrainingArguments(
    output_dir="roberta-classifier-pokemon",
    gradient_accumulation_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # eval_steps=1,
    save_strategy="epoch",
    # save_steps=1,
    logging_strategy="epoch",
    # logging_steps=1,
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.003700,12.739668,0.061871
2,0.005800,12.908293,0.064029
4,0.002800,12.818261,0.061871
6,0.005700,12.900506,0.056835


Saving training

In [18]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TeeA/roberta-classifier-pokemon/commit/02a6d3d21badf770fae8db79a77a9970cfbff502', commit_message='End of training', commit_description='', oid='02a6d3d21badf770fae8db79a77a9970cfbff502', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
tokenizer.push_to_hub("TeeA/roberta-classifier-pokemon")
model.push_to_hub("TeeA/roberta-classifier-pokemon")

CommitInfo(commit_url='https://huggingface.co/TeeA/roberta-classifier-pokemon/commit/8035a89679d6f006f643f95bee80622ba4f980c0', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='8035a89679d6f006f643f95bee80622ba4f980c0', pr_url=None, pr_revision=None, pr_num=None)

# Prediction

In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification


device = "cuda" if torch.cuda.is_available() else "cpu"


tokenizer = AutoTokenizer.from_pretrained("TeeA/roberta-classifier-pokemon")
model = RobertaForSequenceClassification.from_pretrained("TeeA/roberta-classifier-pokemon").to(device)

In [ ]:
description = """It is a quadrupedal equine Pokémon with a golden coat and a flowing mane and tail made of fire. It has a white muzzle, a red nose, and large, black eyes. Its hooves are also black."""

inputs = tokenizer(description, return_tensors="pt").to(device)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'Flareon'